In [2]:
from sklearn.preprocessing import FunctionTransformer

from reskit.norms import binar_norm, wbysqdist
from reskit.norms import spectral_norm

from reskit.features import degrees,  pagerank

from reskit.core import Transformer, Pipeliner

from sklearn.feature_selection import VarianceThreshold

from sklearn.preprocessing import MinMaxScaler

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier 
from xgboost import XGBClassifier

from sklearn.model_selection import StratifiedKFold

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

import os
import pandas as pd
import numpy as np

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

def orig(x):
    return x

In [3]:
def get_autism(path_to_read='Data/dti/', distances=True):
    def get_autism_distances(loc_name):
        with open(loc_name, 'r') as f:
            read_data = f.readlines()

        read_data = pd.DataFrame(
            np.array([np.array(item[:-1].split()).astype(int) for item in read_data]))

        return read_data

    def get_distance_matrix(coords):
        if type(coords) == pd.core.frame.DataFrame:
            coords = coords.values
        elif type(coords) != np.ndarray:
            print('Provide either pandas df or numpy array!')
            return -1

        shape = len(coords)
        dist_matrix = np.zeros((shape, shape))
        del shape
        for i in range(len(coords)):
            for j in range(i + 1, len(coords)):
                dist_matrix[i, j] = np.linalg.norm(coords[i, :] - coords[j, :])
                dist_matrix[j, i] = dist_matrix[i, j]
        return dist_matrix

    target_vector = []  # this will be a target vector (diagnosis)
    matrices = []  # this will be a list of connectomes
    all_files = sorted(os.listdir(path_to_read))
    matrix_files = [
        item for item in all_files if 'DTI_connectivity' in item and 'All' not in item]
    distance_files = [
        item for item in all_files if 'DTI_region_xyz_centers' in item and 'All' not in item]

    # for each file in a sorted (!) list of files:
    for filename in matrix_files:

        A_dataframe = pd.read_csv(
            path_to_read + filename, sep='   ', header=None, engine='python')
        A = A_dataframe.values  # we will use a list of numpy arrays, NOT pandas dataframes
        matrices.append(A)# append a matrix to our list
        if "ASD" in filename:
            target_vector.append(1)
        elif "TD" in filename:
            target_vector.append(0)
    asd_dict = {}
    asd_dict['X'] = np.array(matrices)
    asd_dict['y'] = np.array(target_vector)
    if distances:
        dist_matrix_list = []
        for item in distance_files:
            # print(item)
            cur_coord = get_autism_distances(path_to_read + item)
            cur_dist_mtx = get_distance_matrix(cur_coord)
            dist_matrix_list += [cur_dist_mtx]

        asd_dict['dist'] = np.array(dist_matrix_list)

    return asd_dict

In [4]:
import matrix_eig as me

# Train

In [16]:
data = [('UCLAsource', Transformer(get_autism))]

weighters = [('origW', Transformer(orig))]

normalizers = [('origN', Transformer(orig)),
               ('spectral', Transformer(spectral_norm))]

featurizers = [('origF', Transformer(me.orig_vec, collect=['X_vec'])),
               ('low230', Transformer(me.matrix_eig_230, collect=['X_vec'])),
               ('low240', Transformer(me.matrix_eig_240, collect=['X_vec'])),
               ('low250', Transformer(me.matrix_eig_250, collect=['X_vec'])),
               ('low260', Transformer(me.matrix_eig_260, collect=['X_vec']))]

selectors = [('var_threshold', VarianceThreshold())]

scalers = [('minmax', MinMaxScaler())]

classifiers = [('LR_orig' , LogisticRegression()),
               ('LR_binar', LogisticRegression())]

steps = [('Data', data),
         ('Weighters', weighters),
         ('Normalizers', normalizers),
         ('Featurizers', featurizers),
         ('Selectors', selectors),
         ('Scalers', scalers),
         ('Classifiers', classifiers)]

param_grid = dict(
     LR_orig =dict(penalty=['l1'], C=[0.4], max_iter=[50]),
     LR_binar=dict(penalty=['l2'], C=[0.6], max_iter=[50])
    )

banned_combos = [('LR_orig', 'origN'),
                 ('LR_binar', 'spectral'),
                 ('LR_orig', 'low250'),
                 ('LR_orig', 'low260'),
                 ('LR_binar', 'low230'),
                 ('LR_binar', 'low240')]


In [17]:
grid_cv = StratifiedKFold(n_splits=10,
                          shuffle=True,
                         random_state=0)

eval_cv = StratifiedKFold(n_splits=10,
                             shuffle=True,
                            random_state=1)

pipe = Pipeliner(steps, eval_cv=eval_cv, grid_cv=grid_cv, param_grid=param_grid, banned_combos=banned_combos)
pipe.plan_table

,Data,Weighters,Normalizers,Featurizers,Selectors,Scalers,Classifiers
0,UCLAsource,origW,origN,origF,var_threshold,minmax,LR_binar
1,UCLAsource,origW,origN,low250,var_threshold,minmax,LR_binar
2,UCLAsource,origW,origN,low260,var_threshold,minmax,LR_binar
3,UCLAsource,origW,spectral,origF,var_threshold,minmax,LR_orig
4,UCLAsource,origW,spectral,low230,var_threshold,minmax,LR_orig
5,UCLAsource,origW,spectral,low240,var_threshold,minmax,LR_orig


In [14]:
num_epoch = 30
for epoch in range(num_epoch):

    grid_cv = StratifiedKFold(n_splits=10,
                              shuffle=True,
                              #random_state=0
                             )

    eval_cv = StratifiedKFold(n_splits=10,
                              shuffle=True,
                              random_state=1)

    
    
    pipe = Pipeliner(steps, eval_cv=eval_cv, grid_cv=grid_cv, param_grid=param_grid, banned_combos=banned_combos)
    #pipe.plan_table
    
    pipe.get_results('Data/dti/', caching_steps=['Data', 'Weighters', 'Normalizers', 'Featurizers'],
                 scoring=['roc_auc'], results_file = "LR_data_of_boxplot/results_" + str(epoch) + '.csv')

Removed previous results file -- LR_data_of_boxplot/results_0.csv.
Line: 1/6
Line: 2/6
Line: 3/6
Line: 4/6
Line: 5/6
Line: 6/6
No previous results found.
Line: 1/6
Line: 2/6
Line: 3/6
Line: 4/6
Line: 5/6
Line: 6/6
No previous results found.
Line: 1/6
Line: 2/6
Line: 3/6
Line: 4/6
Line: 5/6
Line: 6/6
No previous results found.
Line: 1/6
Line: 2/6
Line: 3/6
Line: 4/6
Line: 5/6
Line: 6/6
No previous results found.
Line: 1/6
Line: 2/6
Line: 3/6
Line: 4/6
Line: 5/6
Line: 6/6
No previous results found.
Line: 1/6
Line: 2/6
Line: 3/6
Line: 4/6
Line: 5/6
Line: 6/6
No previous results found.
Line: 1/6
Line: 2/6
Line: 3/6
Line: 4/6
Line: 5/6
Line: 6/6
No previous results found.
Line: 1/6
Line: 2/6
Line: 3/6
Line: 4/6
Line: 5/6
Line: 6/6
No previous results found.
Line: 1/6
Line: 2/6
Line: 3/6
Line: 4/6
Line: 5/6
Line: 6/6
No previous results found.
Line: 1/6
Line: 2/6
Line: 3/6
Line: 4/6
Line: 5/6
Line: 6/6
No previous results found.
Line: 1/6
Line: 2/6
Line: 3/6
Line: 4/6
Line: 5/6
Line: 6/6
No 